In [2]:
import os
import pandas as pd
import requests, scrapy

In [6]:
data = pd.read_csv(os.path.join("Links", 'teaminfolinks.csv'))

In [13]:
data = data[data['key'] == 'stats'].to_dict(orient='record')

In [64]:
test = {}
for x in data: 
    r = requests.get(x['Link'])
    tables = pd.read_html(r.content)
    table = tables[2]
    stop_index = [1 if 'Unnamed: ' in column  else 0 for idx, column in enumerate(table.columns)].index(1)
    columnNamesFixed = {col:col.replace(" ","").replace("/","").replace(".","") for col in table.iloc[:,:stop_index].columns}
    table.rename(columns = columnNamesFixed, inplace = True)
    final_table = table[:-3].iloc[:,:stop_index]
    test[x['Stat']] = final_table

In [19]:
def table_cleaner(html, target_table, trim_n_rows = 0):
    """Read HTML string and return a table
       html : HTML string
       target_table : Target table number
       trim_n_rows : Trim X number of rows from the end of the table (enter as positive number)
    """
    tables = pd.read_html(html)
    table = tables[target_table]
    stop_index = [1 if 'Unnamed: ' in column  else 0 for idx, column in enumerate(table.columns)].index(1)
    columnNamesFixed = {col:col.replace(" ","").replace("/","").replace(".","") for col in table.iloc[:,:stop_index].columns}
    table.rename(columns = columnNamesFixed, inplace = True)
    return table[:-trim_n_rows].iloc[:,:stop_index]
    

In [14]:
r = requests.get(data[0]['link'])

In [15]:
tables = pd.read_html(r.content)

In [16]:
len(tables)

3

In [17]:
tables[2]

,Jersey,Player,Yr,Pos,GP,GS,G,Rush Attempts,Rush Net Yards,Rush YdsGained,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
0,51,"Pierce, Jordan",NaN,NaN,12,3,12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25,"Ladipo, Roland",Fr,DB,12,0,12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,"Robinette, Jalen",Fr,WR,12,2,12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72,"Rochell, Matt",So,OL,12,12,12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,82,"Link, Keith",Jr,TE,12,0,12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7,"Brown, Garrett",So,WR,12,2,12,8.0,34.0,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,20,"McHenry, Gavin",So,DB,12,12,12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5,"Walker, Dexter",So,DB,12,8,12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,85,"Hendricks, Marcus",Jr,TE,12,6,12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,34,"Kohn, Demario",Jr,RB,12,0,12,5.0,10.0,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
table_cleaner(r.content, 2, 1)

,Jersey,Player,Yr,Pos,GP,GS,G,RushAttempts,RushNetYards,RushYdsGained,YdsRush,RushYdsLost,RushTDs,RushYdsG,RushLong
0,51,"Pierce, Jordan",NaN,NaN,12,3,12,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
1,25,"Ladipo, Roland",Fr,DB,12,0,12,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
2,9,"Robinette, Jalen",Fr,WR,12,2,12,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
3,72,"Rochell, Matt",So,OL,12,12,12,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
4,82,"Link, Keith",Jr,TE,12,0,12,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
5,7,"Brown, Garrett",So,WR,12,2,12,8.0,34.0,36,4.25,2.0,1,2.83,33
6,20,"McHenry, Gavin",So,DB,12,12,12,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
7,5,"Walker, Dexter",So,DB,12,8,12,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
8,85,"Hendricks, Marcus",Jr,TE,12,6,12,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
9,34,"Kohn, Demario",Jr,RB,12,0,12,5.0,10.0,14,2.00,4.0,NaN,0.83,10


Index(['Jersey', 'Player', 'Yr', 'Pos', 'GP', 'GS', 'G', 'PassAttempts',
       'Completions', 'Interceptions', 'PassYards', 'PassTDs', 'PassEff',
       'CompletionsPerGame', 'YdsPerCompletion', 'CompletionPct',
       'PassYardsG'],
      dtype='object')

In [111]:
from scrapy.item import DictItem, Field

def create_item_class(class_name,field_list):
    field_dict = {}
    for field_name in field_list:
        field_dict[field_name] = Field()

    return type(class_name, (DictItem,), {'fields':field_dict})


In [149]:
field_list = test['Participation'].columns
DynamicItem = create_item_class('passing', field_list)
passing = DynamicItem()

In [151]:
passing.fields

{'G': {}, 'GP': {}, 'GS': {}, 'Jersey': {}, 'Player': {}, 'Pos': {}, 'Yr': {}}

In [157]:
class item2(scrapy.Item):
    """Scrapy Item Team History"""
    Jersey = scrapy.Field()
    Player = scrapy.Field()
    Yr = scrapy.Field()
    Pos = scrapy.Field()
    GP = scrapy.Field()
    GS = scrapy.Field()
    G = scrapy.Field()

In [154]:
for record in test['Participation'].to_dict(orient='record'):
    for k, v in record.items(): 
        passing[k] = v

In [160]:
for record in test['Participation'].to_dict(orient='record')[:1]:
    x = item2(record)
    print(x.__dict__)

{'_values': {'Jersey': '87', 'Player': 'Hansen, Alexander', 'Yr': 'Jr', 'Pos': 'DL', 'GP': '13', 'GS': '13', 'G': 13}}


In [155]:
passing.__dict__

{'_values': {'G': 1,
  'GP': '1',
  'GS': '0',
  'Jersey': '90',
  'Player': 'Allen, Reed',
  'Pos': 'DL',
  'Yr': 'Jr'}}

In [156]:
item2

__main__.item2